### 앞서 전처리 과정 + 모델 선정 과정을 거친 최종 모델로 RPM 모델 제작 

#### 기존의 전처리 방법과 다른 점
: 모델 및 스케일러를 미리 선정할지, 성적 위주로 선정할지가 다름  
1. 기존 전처리 방법 : 모델 및 스케일러를 미리 선정
2. 스케일러 : StandardScaler; 데이터의 분포가 정규분포를 따르기 때문에, 또 이상치가 적기 때문에 사용
3. 모델 : 각 머신러닝, 앙상블, 부스팅 -> linear Regression; 성적을 비교한 후 fine_tuning을 통해 최적의 모델 선정
4. 단점 : 데이터나 모델에 대한 이해가 부족할 때 더 나은 방법을 놓칠 수 있음

#### 이번 전처리 방법 : 성적 위주로 선정
1. 세 가지 스케일러와 다수의 모델을 비교하고 최적의 모델을 선정
2. 데이터의 규모가 작아 가능한 방법
3. 또한 cross_val_score를 통해 성적을 비교하므로, 성적을 비교하는 것이 더욱 정확함
4. 참고 정보 : 
   1. StandardScaler : 정규 분포 / 평균 0, 분산 1 / 이상치를 신경쓰지 않음
   2. MinMaxScaler : 최소 0, 최대 1 / 이상치에 민감: 이상치가 많고 반영하고 싶을 때 사용
   3. RobustScaler : 중앙값 0, IQR 1 / 이상치에 둔감: 이상치가 많고 배제하고 싶을 때 사용

---

In [1]:
from model_selection import Scale_Prediction as sp
from bayesian_optimizer import bayes_SVR_RFR

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the data
data = pd.read_csv('../DATA/raw_2023051820231018_경대기업맞춤형.csv')
train_data, test_data = sp().preps(data)

# 2 ~ 4 scale
train_data = sp().scale_2_4(train_data)
test_data = sp().scale_2_4(test_data)

# Split
X_train = train_data.drop('scale_pv', axis=1)
y_train = train_data['scale_pv']
X_test = test_data.drop('scale_pv', axis=1)
y_test = test_data['scale_pv']

# Modeling
# basic_modeling(X_train, y_train, X_test, y_test)


### 기본 모델링 결과
1. 모델 : linear regression, Random Forest, LightGBM (각각 머신러닝, 앙상블, 부스팅 모델)
2. 평가지표 : MAE, MAPE, R2, mean_diff
3. scaler의 영향은 크게 없어 보임, MAPE는 standard, diff에선 

In [3]:
# Bayesian Optimization은 모델 선정 후 사용
scores_df = sp().basic_modeling(X_train, y_train, X_test, y_test)
scores_df.head()

LinearRegression with None. LinearRegression with StandardScaler. LinearRegression with MinMaxScaler. LinearRegression with RobustScaler. Lasso with None. Lasso with StandardScaler. Lasso with MinMaxScaler. Lasso with RobustScaler. Ridge with None. Ridge with StandardScaler. Ridge with MinMaxScaler. Ridge with RobustScaler. BayesianRidge with None. BayesianRidge with StandardScaler. BayesianRidge with MinMaxScaler. BayesianRidge with RobustScaler. ElasticNet with None. ElasticNet with StandardScaler. ElasticNet with MinMaxScaler. ElasticNet with RobustScaler. RandomForestRegressor with None. RandomForestRegressor with StandardScaler. RandomForestRegressor with MinMaxScaler. RandomForestRegressor with RobustScaler. GradientBoostingRegressor with None. GradientBoostingRegressor with StandardScaler. GradientBoostingRegressor with MinMaxScaler. GradientBoostingRegressor with RobustScaler. AdaBoostRegressor with None. AdaBoostRegressor with StandardScaler. AdaBoostRegressor with MinMaxScale

,model,scaler,mae,mape,mean_diff,std_diff,min_diff,25%_diff,50%_diff,75%_diff,max_diff,top_features
0,LinearRegression,None,0.025878,0.852407,0.009337,0.030945,-0.182137,-0.008410,0.009348,0.025435,0.218106,n_temp_pv k_rpm_pv s_temp_pv
1,LinearRegression,StandardScaler,0.025878,0.852407,0.009337,0.030945,-0.182137,-0.008410,0.009348,0.025435,0.218106,n_temp_pv s_temp_pv c_temp_pv
2,LinearRegression,MinMaxScaler,0.025878,0.852407,0.009337,0.030945,-0.182137,-0.008410,0.009348,0.025435,0.218106,n_temp_pv s_temp_pv c_temp_pv
3,LinearRegression,RobustScaler,0.025878,0.852407,0.009337,0.030945,-0.182137,-0.008410,0.009348,0.025435,0.218106,n_temp_pv c_temp_pv s_temp_pv
4,Lasso,None,0.026316,0.866306,0.005966,0.032548,-0.191614,-0.011614,0.008386,0.018386,0.238386,s_temp_pv n_temp_pv k_rpm_pv


In [4]:
scores_df.to_csv('../DATA/scores_df.csv', index=False)
sp().models

[('LinearRegression', LinearRegression()),
 ('Lasso', Lasso()),
 ('Ridge', Ridge()),
 ('BayesianRidge', BayesianRidge()),
 ('ElasticNet', ElasticNet()),
 ('RandomForestRegressor', RandomForestRegressor()),
 ('GradientBoostingRegressor', GradientBoostingRegressor()),
 ('AdaBoostRegressor', AdaBoostRegressor()),
 ('LGBMRegressor', LGBMRegressor()),
 ('ARDRegression', ARDRegression()),
 ('HuberRegressor', HuberRegressor())]

### scores_df.csv 저장 완료